# Use Custom Template Notebook

In [2]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#Modeling Libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier

from xgboost import XGBClassifier
#from catboost import CatBoostClassifier

from joblib import dump, load
import requests

# Initialize Snowflake Session using Default connection

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
# To read a specific dataset published from a snowflake connection
#df = snowflake.get_dataframe("AUTO_INSURANCE_CLAIMS_DATA")
#df = snowflake.get_dataframe("AUTO_INSURANCE_CLAIMS_DATA_PRODUCT")

table_name = 'AUTO_INSURANCE_CLAIMS_DATA'

In [5]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [6]:
df = sf_df.to_pandas()

In [7]:
df.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,POLICY_BIND_DATE_CUSTOM
0,132,36,460979,14-08-2005,CT,100/300,712,800,0,431104,...,Police Report Available,65327,11917,9753,43657,Ford,Fusion,2004,Fraud Reported,2005-08-14
1,51,28,460980,30-03-1995,CT,250/500,1230,1708,11098942,473705,...,No Police Report Available,76311,487,4689,71135,Suburu,Legacy,1995,Fraud Reported,1995-03-30
2,38,26,460981,05-01-2002,CT,100/300,1697,2044,11191554,612316,...,Police Report Available,41467,559,1085,39823,BMW,X6,1995,No Fraud Reported,2002-01-05
3,37,48,460982,30-06-2010,CT,250/500,1858,2293,15946216,447560,...,No Police Report Available,58415,17898,968,39549,Ford,F150,2007,Fraud Reported,2010-06-30
4,305,53,460983,19-01-1993,CT,250/500,579,826,0,608758,...,No Police Report Available,73157,325,13230,59602,Saab,92x,2008,No Fraud Reported,1993-01-19


In [8]:
insurance_claim =  df.copy()

In [11]:
insurance_claim = insurance_claim[['MONTHS_AS_CUSTOMER', 'CUSTOMER_AGE', 'POLICY_NUMBER',
       'POLICY_BIND_DATE', 'POLICY_STATE', 'POLICY_CSL', 'POLICY_DEDUCTABLE',
       'POLICY_ANNUAL_PREMIUM', 'UMBRELLA_LIMIT', 'INSURED_ZIP', 'INSURED_SEX',
       'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION', 'INSURED_HOBBIES',
       'INSURED_RELATIONSHIP', 'CAPITAL_GAINS', 'CAPITAL_LOSS',
       'INCIDENT_DATE', 'INCIDENT_TYPE', 'COLLISION_TYPE', 'INCIDENT_SEVERITY',
       'AUTHORITIES_CONTACTED', 'INCIDENT_STATE', 'INCIDENT_CITY',
       'INCIDENT_LOCATION', 'INCIDENT_HOUR_OF_THE_DAY', 'INCIDENT_TIME_OF_DAY',
       'NUMBER_OF_VEHICLES_INVOLVED', 'PROPERTY_DAMAGE', 'BODILY_INJURIES',
       'WITNESSES', 'POLICE_REPORT_AVAILABLE', 'TOTAL_CLAIM_AMOUNT_PAID',
       'INJURY_CLAIM', 'PROPERTY_CLAIM', 'VEHICLE_CLAIM', 'AUTO_MAKE',
       'AUTO_MODEL', 'AUTO_YEAR', 'FRAUD_REPORTED']]

In [12]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [13]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [14]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [15]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [34]:
df = insurance_claim.copy()

In [35]:
actual_inference = df.copy()
actual_inference.drop('fraud_reported', axis = 1, inplace=True)

In [36]:
actual_inference.head()

,months_as_customer,customer_age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,132,36,460979,14-08-2005,CT,100/300,712,800,0,431104,...,1,0,Police Report Available,65327,11917,9753,43657,Ford,Fusion,2004
1,51,28,460980,30-03-1995,CT,250/500,1230,1708,11098942,473705,...,0,1,No Police Report Available,76311,487,4689,71135,Suburu,Legacy,1995
2,38,26,460981,05-01-2002,CT,100/300,1697,2044,11191554,612316,...,0,2,Police Report Available,41467,559,1085,39823,BMW,X6,1995
3,37,48,460982,30-06-2010,CT,250/500,1858,2293,15946216,447560,...,2,1,No Police Report Available,58415,17898,968,39549,Ford,F150,2007
4,305,53,460983,19-01-1993,CT,250/500,579,826,0,608758,...,0,3,No Police Report Available,73157,325,13230,59602,Saab,92x,2008


In [37]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [38]:
df['incident_hour_of_the_day'] = df['incident_hour_of_the_day'].astype(str).astype(int)

In [49]:
df['incident_hour_of_the_day'].dtype

dtype('int64')

In [39]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [40]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [41]:
cat_df.columns

Index(['policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [42]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

policy_csl: 
['100/300' '250/500' '500/1000']

insured_sex: 
['MALE' 'FEMALE']

insured_education_level: 
['Associate' 'College' 'Masters' 'JD' 'MD' 'High School' 'PhD']

insured_occupation: 
['protective-serv' 'exec-managerial' 'farming-fishing' 'sales'
 'machine-op-inspct' 'craft-repair' 'prof-specialty' 'armed-forces'
 'priv-house-serv' 'other-service' 'transport-moving' 'tech-support'
 'handlers-cleaners' 'adm-clerical']

insured_relationship: 
['other-relative' 'own-child' 'wife' 'husband' 'unmarried' 'not-in-family']

incident_type: 
['Single Vehicle Collision' 'Multi-vehicle Collision' 'Parked Car'
 'Vehicle Theft']

collision_type: 
['Rear Collision' 'Side Collision' 'Details not Available'
 'Front Collision']

incident_severity: 
['Major Damage' 'Minor Damage' 'Total Loss']

authorities_contacted: 
['Police' 'Ambulance' 'Other' 'Fire' 'None']

incident_time_of_day: 
['Morning to Noon' 'Night Time' 'Afternoon Hours' 'Early Morning Hours']

property_damage: 
['Property Damage' '

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232025 entries, 0 to 232024
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   months_as_customer           232025 non-null  int16 
 1   customer_age                 232025 non-null  int8  
 2   policy_csl                   232025 non-null  object
 3   policy_deductable            232025 non-null  int16 
 4   policy_annual_premium        232025 non-null  int16 
 5   umbrella_limit               232025 non-null  int32 
 6   insured_sex                  232025 non-null  object
 7   insured_education_level      232025 non-null  object
 8   insured_occupation           232025 non-null  object
 9   insured_relationship         232025 non-null  object
 10  capital_gains                232025 non-null  int32 
 11  capital_loss                 232025 non-null  int32 
 12  incident_type                232025 non-null  object
 13  collision_type

In [43]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [50]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64','int64'])

In [51]:
num_df.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')

In [52]:
cat_df.columns

Index(['policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed-forces',
       'insured_occupation_craft-repair', 'insured_occupation_exec-managerial',
       'insured_occupation_farming-fishing',
       'insured_occupation_handlers-cleaners',
       'insured_occupation_machine-op-inspct',
       'insured_occupation_other-service',
       'insured_occupation_priv-house-serv',
       'insured_occupation_prof-specialty',
       'insured_occupation_protective-serv', 'insured_occupation_sales',
       'insured_occupation_tech-support',
       'insured_occupation_transport-moving',
       'insured_relationship_not-in-family',
       'insured_relationship_other-relative', 'insured_relationship_own-child',
       'insured_relationship

In [53]:
# combining the Numerical and Categorical dataframes to get the final dataset

X = pd.concat([num_df, cat_df], axis = 1)

In [90]:
X.columns = X.columns.str.replace('/', '_')

In [91]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [92]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [93]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [94]:
# Scaling the numeric values in the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [95]:
scaled_data_test = scaler.transform(num_df_test)

In [96]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
52464,-1.074365,1.171567,0.524636,1.253019,1.455436,-1.028454,-0.678716,-1.219944,-0.448911,-1.034112,-0.674831,0.942329
35935,0.772600,0.324193,-0.044597,-1.709932,1.278714,1.357754,-0.678716,-1.219944,-1.344550,-1.002203,-0.971956,-0.358409
215508,0.717631,0.477243,-0.847920,0.166187,-1.274129,1.357754,-0.678716,1.225438,1.342367,1.050581,-0.950197,1.146338
114957,-0.326784,-1.370555,0.005848,1.046182,0.179183,0.505537,0.745655,1.225438,-0.448911,1.880064,1.431005,-0.769038
20347,0.003031,2.095466,1.390130,0.753946,-1.379817,0.846423,-0.678716,1.225438,1.342367,1.179475,-0.937599,-1.077312


In [97]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
58313,-0.920452,-0.263743,-0.564874,0.659519,0.578775,1.357754,-0.678716,-1.219944,0.446728,-1.002203,-0.964703,-0.670696
18972,-1.294242,0.245802,0.377463,-0.563230,1.401707,-0.346680,-0.678716,-1.219944,0.446728,-0.999264,2.129982,-0.331021
26999,-1.107347,-1.628127,-0.300425,0.429608,0.927039,-2.562444,0.745655,0.002747,-1.344550,1.441181,1.938473,1.119077
226298,-0.513679,-1.325760,-1.760339,-1.018033,0.420033,1.187310,-0.678716,-1.219944,1.342367,-1.026135,1.092397,0.544821
198422,0.156945,1.423539,0.988597,0.491897,0.636671,-0.687567,-0.678716,0.002747,-1.344550,0.723658,-1.071592,0.265973


In [80]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [81]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [82]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [83]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Decision Tree

In [84]:
#from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [85]:
y_pred = dtc.predict(X_test)

In [86]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9942937921285362
[[ 8594   165]
 [  166 49082]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8759
No Fraud Reported       1.00      1.00      1.00     49248

         accuracy                           0.99     58007
        macro avg       0.99      0.99      0.99     58007
     weighted avg       0.99      0.99      0.99     58007



In [87]:
from fosforml import register_model

In [89]:
X.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim',
       'policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed-forces',
       'insured_occupation_craft-repair', 'insured_occupation_exec-managerial',
       'insured_occupation_farming-fishing',
       'insured_occupation_handlers-cleaners',
       'insured_occupation_machine-op-inspct',
       'insured_occupation_other-service',
       'insured_occupation_priv-house-serv',
       'insured_occupati

In [88]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=dtc, 
    session=my_session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    name="Claims_Fraud_DTree_Classifier",
    description="Insurance claim's model trained for Fraud Classification",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["fosforml==1.6."]
)

"error, invalid inputs : Column name 'policy_csl_250/500' is invalid. Column names must start with a letter or underscore and contain only letters, numbers, underscores and $ signs."

# Version 2 of Decision Tree Fraud Classifier Model

In [125]:
dtc = DecisionTreeClassifier(random_state=7)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=7)

In [126]:
y_pred = dtc.predict(X_test)
y_prob = dtc.predict_proba(X_test)[:,1]

In [127]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9941263851657076
[[ 8305   156]
 [  188 49918]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8461
No Fraud Reported       1.00      1.00      1.00     50106

         accuracy                           0.99     58567
        macro avg       0.99      0.99      0.99     58567
     weighted avg       0.99      0.99      0.99     58567



In [128]:
## registering the model in Fosfor Insight Designer using same name.
model_reg = register_model(dtc, 
               score, 
               name="Claims_Fraud_DTree_Classifier", 
               description="Insurance claim's model trained for Fraud Classification",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred, 
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
